In [1]:
# import sys
# from pathlib import Path

# # Get the absolute path to the project root
# notebook_path = Path('/Users/nirmal/Documents/np_research/ManGo_code/ManGo')
# project_root = str(notebook_path)

# # Add the project root to Python path if it's not already there
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

# # Now you can import your module
# from src.models.mg_rig_wrp import GnosticRobustRegressor

# # Verify the import worked
# # print(f"Project root: {project_root}")
# # print(f"Available at: {RobustRegressor.__module__}")

In [3]:
import pandas as pd
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from machinegnostics.models.regression import PolynomialRegressor
import numpy as np
# Create and train the model
model = PolynomialRegressor(degree=3, mg_loss='hi', verbose=True)
X = np.random.rand(100, 2)
y = np.random.rand(100)
model.fit(X, y)

# Create an input example
input_example = pd.DataFrame(X, columns=["feature1", "feature2"])

# Infer the model signature
signature = infer_signature(input_example, model.predict( input_example))

# Log the model to MLflow with input_example and signature
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="robust_regressor",
        python_model=model,
        registered_model_name="GnosticRobustRegressor",
        input_example=input_example,
        signature=signature
    )

2025-09-23 21:30:44,609 | PolynomialRegressor | INFO | PolynomialRegressor initialized:
2025-09-23 21:30:44,612 | PolynomialRegressor | INFO | DataProcessLayerBase initialized.
2025-09-23 21:30:44,614 | PolynomialRegressor | INFO | DataProcessRobustRegressor initialized.
2025-09-23 21:30:44,615 | PolynomialRegressor | DEBUG | PolynomialRegressor initialized.
2025-09-23 21:30:44,617 | PolynomialRegressor | INFO | Starting fit process.
2025-09-23 21:30:44,617 | PolynomialRegressor | INFO | Starting fit process for DataProcessRobustRegressor.
2025-09-23 21:30:44,618 | PolynomialRegressor | INFO | Starting fit input/output processing.
2025-09-23 21:30:44,618 | PolynomialRegressor | INFO | Checking input X of type: <class 'numpy.ndarray'>
2025-09-23 21:30:44,618 | PolynomialRegressor | INFO | Identifying and converting data of type: <class 'numpy.ndarray'>
2025-09-23 21:30:44,618 | PolynomialRegressor | INFO | Checking target y of type: <class 'numpy.ndarray'>
2025-09-23 21:30:44,619 | Poly

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Convert history to DataFrame if it's a list of dicts
if isinstance(model._history, list):
    history_df = pd.DataFrame(model._history)
else:
    history_df = pd.DataFrame([model._history])

# Plot h_loss if present
if 'h_loss' in history_df.columns and history_df['h_loss'].notnull().any():
    plt.plot(history_df['iteration'], history_df['h_loss'], label='h_loss')
    plt.xlabel('Iteration')
    plt.ylabel('h_loss')
    plt.title('Gnostic Loss (h_loss) over Iterations')
    plt.legend()
    plt.show()

# Plot rentropy if present
if 'rentropy' in history_df.columns and history_df['rentropy'].notnull().any():
    plt.plot(history_df['iteration'], history_df['rentropy'], label='rentropy', color='orange')
    plt.xlabel('Iteration')
    plt.ylabel('Rentropy')
    plt.title('Residual Entropy over Iterations')
    plt.legend()
    plt.show()

# Plot coefficients norm if present
if 'coefficients' in history_df.columns and history_df['coefficients'].notnull().any():
    coef_norms = history_df['coefficients'].apply(lambda x: np.linalg.norm(x) if x is not None else np.nan)
    plt.plot(history_df['iteration'], coef_norms, label='||coefficients||', color='green')
    plt.xlabel('Iteration')
    plt.ylabel('Coefficient Norm')
    plt.title('Coefficient Norm over Iterations')
    plt.legend()
    plt.show()

# Plot mean weights if present
if 'weights' in history_df.columns and history_df['weights'].notnull().any():
    mean_weights = history_df['weights'].apply(lambda x: np.mean(x) if x is not None else np.nan)
    plt.plot(history_df['iteration'], mean_weights, label='Mean Weights', color='red')
    plt.xlabel('Iteration')
    plt.ylabel('Mean Weight')
    plt.title('Mean Sample Weight over Iterations')
    plt.legend()
    plt.show()

In [ ]:
# Load the model from MLflow
loaded_model = mlflow.pyfunc.load_model("models:/GnosticRobustRegressor/1")

# Use the model for predictions
import pandas as pd
X_test = pd.DataFrame(np.random.rand(10, 2), columns=["feature1", "feature2"])
predictions = loaded_model.predict(X_test)
print(predictions)

In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from machinegnostics.models import RobustRegressor  # Replace with your actual import path

# Step 1: Set up MLflow experiment
# mlflow.set_experiment("Robust Regressor Experiment")

# Step 2: Generate synthetic data
X = np.random.rand(100, 2)  # 100 samples, 2 features
y = 2.5 * X[:, 0] + 1.5 * X[:, 1] + np.random.normal(scale=0.1, size=100)  # Linear relationship with noise

# Step 3: Train the model
model = RobustRegressor(degree=3, mg_loss='hi', verbose=True)
model.fit(X, y)

# Step 4: Create an input example and infer the model signature
input_example = pd.DataFrame(X, columns=["feature1", "feature2"])
signature = infer_signature(model_input=input_example, model_output=y)

# Step 5: Log the model, parameters, and metrics to MLflow
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("degree", 3)
    mlflow.log_param("mg_loss", "hi")
    mlflow.log_param("verbose", True)

    # Log metrics (example: training RMSE)
    y_pred = model.predict(input_example)
    rmse = np.sqrt(np.mean((y - y_pred) ** 2))
    mlflow.log_metric("train_rmse", rmse)

    # Safely log rentropy and h_loss if available and not None
    rentropy = None
    h_loss = None
    if hasattr(model, "_history"):
        hist = model._history
        # If history is a dict of lists
        if isinstance(hist, dict):
            if "rentropy" in hist and hist["rentropy"] and hist["rentropy"][-1] is not None:
                rentropy = hist["rentropy"][-1]
            if "h_loss" in hist and hist["h_loss"] and hist["h_loss"][-1] is not None:
                h_loss = hist["h_loss"][-1]
        # If history is a list of dicts
        elif isinstance(hist, list) and len(hist) > 0:
            if "rentropy" in hist[-1] and hist[-1]["rentropy"] is not None:
                rentropy = hist[-1]["rentropy"]
            if "h_loss" in hist[-1] and hist[-1]["h_loss"] is not None:
                h_loss = hist[-1]["h_loss"]

    if rentropy is not None:
        mlflow.log_metric('train_rentropy', rentropy)
    if h_loss is not None:
        mlflow.log_metric('train_h_loss', h_loss)

    # Log the model
    mlflow.pyfunc.log_model(
        artifact_path="robust_regressor",
        python_model=model,
        registered_model_name="LogisticRobustRegressor",
        input_example=input_example,
        signature=signature
    )

In [ ]:
# save model
model.save_model("robust_regressor_model.pkl")

In [ ]:
# read model and make predictions
loaded_model = model.load_model("robust_regressor_model.pkl")
# Use the loaded model for predictions
X_test = pd.DataFrame(np.random.rand(10, 2), columns=["feature1", "feature2"])
predictions = loaded_model.predict(X_test)
print(predictions)

In [ ]:
# Step 6: Load the model from MLflow
loaded_model = mlflow.pyfunc.load_model("models:/LogisticRobustRegressor/4")

# Step 7: Use the loaded model for predictions
# X_test = pd.DataFrame(np.random.rand(10, 2), columns=["feature1", "feature2"])
predictions = loaded_model.predict(X_test)
print("Predictions:")
print(predictions)